<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_9_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,13,insomnia
2,60,cigarette withdrawal
3,85,tremor
4,121,itches


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-9.test.csv",header=None)
test.head()

,0,1
0,62,feeling proper psycho
1,74,useless without
2,85,tremor
3,91,jitters
4,97,screwing with my short term memory


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.226073,4.636035,0.236161,16:46


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.519253,4.345853,0.263257,16:35


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.222083,4.310462,0.266993,16:51


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.179980,4.285136,0.268925,16:50


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.135608,6.122095,0.175000,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.210918,4.813710,0.191667,00:29


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.707353,4.524813,0.266667,01:08


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.447899,4.115350,0.300000,01:12
1,5.238775,4.043454,0.275000,01:10


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.989370,3.822242,0.325000,01:14
1,4.822857,3.745217,0.333333,01:12


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.618326,3.611442,0.391667,01:15
1,4.456720,3.560879,0.408333,01:13


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.304676,3.514281,0.391667,01:12
1,4.153345,3.457827,0.383333,01:15


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.956273,3.434319,0.433333,01:15
1,3.842558,3.395038,0.408333,01:10


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.660463,3.359578,0.433333,01:09
1,3.622187,3.430465,0.400000,01:15
2,3.450294,3.363256,0.383333,01:09
3,3.303930,3.328437,0.408333,01:12


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.188052,3.291703,0.408333,01:12
1,3.148846,3.321659,0.375000,01:11
2,2.980909,3.317460,0.391667,01:10
3,2.816782,3.296110,0.408333,01:15


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.665996,3.337698,0.425000,01:14
1,2.673884,3.394580,0.425000,01:16
2,2.556024,3.399770,0.433333,01:11
3,2.430873,3.339794,0.433333,01:10


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.351566,3.460400,0.441667,01:09
1,2.339410,3.476300,0.433333,01:10
2,2.199722,3.413475,0.433333,01:10
3,2.114495,3.488206,0.425000,01:12


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.028959,3.486711,0.416667,01:10
1,2.039312,3.531583,0.416667,01:08
2,1.948288,3.479827,0.408333,01:13
3,1.845208,3.476451,0.400000,01:10


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.821059,3.545170,0.450000,01:10
1,1.758655,3.559398,0.425000,01:15
2,1.694718,3.568353,0.450000,01:16
3,1.610306,3.556996,0.466667,01:09


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.638023,3.597983,0.458333,01:10
1,1.631426,3.669998,0.441667,01:12
2,1.572667,3.684664,0.483333,01:16
3,1.541165,3.797249,0.425000,01:09


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.528673,3.785873,0.416667,01:14
1,1.514135,3.795362,0.441667,01:13


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.493244,3.742332,0.391667,01:10
1,1.475488,3.891830,0.375000,01:10


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.394860,3.804618,0.433333,01:14


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[2194, 1099, 1204, 231, 750, 189, 189, 189, 582, 224, 224, 231, 2147, 1936, 233, 2147, 249, 1073, 301, 1925, 1769, 1099, 2113, 749, 749, 749, 818, 231, 2149, 792, 154, 231, 1308, 978, 1884, 1884, 1099, 231, 2023, 815, 947, 974, 974, 974, 974, 249, 978, 978, 989, 1019, 1529, 231, 1718, 231, 418, 1787, 1925, 393, 947, 231, 1798, 2104, 231, 1529, 2066, 1973, 189, 231, 1597, 1973, 1599, 231, 1597, 1432, 1936, 825, 1507, 1526, 1508, 231, 1597, 1597, 1597, 1597, 301, 231, 1669, 561, 231, 1526, 1701, 231, 819, 1526, 1175, 1711, 1718, 1099, 1718, 818, 1718, 1787, 1669, 1253, 1718, 1073, 1073, 1787, 393, 393, 393, 1787, 1521, 1718, 1099, 486, 825, 1599, 1099, 1529, 1942, 231, 231, 1787, 2032, 1787, 1787, 231, 2103, 813, 2147, 2147, 2147, 2147, 1973, 2103, 824, 2194, 1529, 1359, 231, 2194]
[62, 74, 85, 91, 97, 189, 189, 189, 224, 224, 224, 233, 233, 233, 233, 233, 249, 250, 301, 504, 561, 620, 739, 749, 749, 749, 791, 791, 791, 792, 813, 818, 822, 822, 824, 824, 825, 852, 923, 936, 945, 

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

42
0.29577464788732394
